In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

## Package imports and settings 

In [2]:
import pandas as pd
import seaborn as sns
import datetime as dt
import cpi
import requests
import re
from unidecode import unidecode

In [3]:
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

In [4]:
from thefuzz import fuzz
from thefuzz import process


In [5]:
# Right after importing seaborn (could also use 'whitegrid')
sns.set_theme(style='whitegrid', context='talk')

In [6]:
# surpress scientific notation
pd.options.display.float_format = '{:.2f}'.format

# Importing Data

## Lobster Export Data

### Which codes do relate to lobsters?

From : https://www.ic.gc.ca/app/scr/tdst/tdo/crtr.html?grouped=INDIVIDUAL&searchType=KS_CS&naArea=9999&countryList=ALL&toFromCountry=CDN&reportType=TE&timePeriod=5%7CComplete+Years&currency=CDN&productType=HS6&hSelectedCodes=%7C10600%7C30611%7C30612%7C30615%7C30621%7C30622%7C30625%7C30631%7C30632%7C30634%7C30691%7C30692%7C30694%7C160530&runReport=true

Searched for all codes containing word "lobster"

In [7]:
# jlab
path_1605 = 'data/full_data/1605_88Jan_to_23Sep_ATP_PV_Monthly.csv'
df_1605_full = pd.read_csv(path_1605, engine = 'python', skipfooter = 3,index_col=False, header = 1)
df_1605_full.head()

,Period,Commodity,Province,Country,State,Value ($),Quantity,Unit of measure
0,2013-04-01,"1605.21.00 - Shrimps and prawns, prepared or p...",Newfoundland and Labrador,United States,California,195111,29802,Weight in kilograms
1,2013-12-01,"1605.21.00 - Shrimps and prawns, prepared or p...",Newfoundland and Labrador,United States,California,285427,35966,Weight in kilograms
2,2017-10-01,"1605.59.00 - Molluscs, prepared or preserved, nes",Newfoundland and Labrador,United States,California,185609,22500,Weight in kilograms
3,2014-08-01,"1605.10.91 - Crab, snow (Queen), prepared or p...",Newfoundland and Labrador,United States,Alabama,191769,14969,Weight in kilograms
4,2013-01-01,"1605.21.00 - Shrimps and prawns, prepared or p...",Newfoundland and Labrador,United States,California,170470,16053,Weight in kilograms


In [8]:
path_0306 = 'data/full_data/0306-Crust-Jan88-to-Sep23-prov.csv'
df_0306_full = pd.read_csv(path_0306, engine = 'python', skipfooter = 3,index_col=False, header = 1)
df_0306_full.head()

,Period,Commodity,Province,Country,State,Value ($),Quantity,Unit of measure
0,2014-06-01,"0306.16.10 - Cold-water shrimps and prawns, fr...",Newfoundland and Labrador,Greenland,NaN,220915,66528,Weight in kilograms
1,2023-04-01,"0306.14.10 - Crabs, snow (Queen), frozen",Newfoundland and Labrador,United States,California,1111067,61916,Weight in kilograms
2,2015-01-01,"0306.16.10 - Cold-water shrimps and prawns, fr...",Newfoundland and Labrador,Greenland,NaN,753558,174576,Weight in kilograms
3,2015-06-01,"0306.16.10 - Cold-water shrimps and prawns, fr...",Newfoundland and Labrador,Greenland,NaN,883171,199392,Weight in kilograms
4,2015-07-01,"0306.17.10 - Shrimps and prawns, frozen, in sh...",Newfoundland and Labrador,Saint Pierre and Miquelon,NaN,704,45,Weight in kilograms


## Country GDP

In [9]:
# TODO store API key as env variable 
api_key = 'DsPAQbicFL8WYvYihOWoFQ==Vd8zztny3V9w3Q7n'

In [10]:
gdp_data = pd.read_csv('data/gdp-data/gdp_data.csv',engine= 'python', header = 2)
gdp_data

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2791061452.51,2963128491.62,2983798882.68,3092178770.95,3276187709.50,3395793854.75,2610038938.55,3126019385.47,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,21125015452.20,21616228139.04,23506279899.97,28048360188.28,25920665259.89,29472103269.74,...,1006526390112.83,927348469903.11,885176429223.51,1021043200767.01,1007196197587.41,1000834077088.55,927593321647.66,1081998139192.47,1169483734191.46,NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,537777811.11,548888895.56,546666677.78,751111191.11,800000044.44,1006666637.78,...,20550582746.84,19998143635.87,18019554403.45,18896353155.88,18418860354.42,18904502222.21,20143451705.75,14583135236.57,NaN,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,10447637852.92,11173212080.39,11990534017.87,12727688165.45,13898109284.02,14929792387.52,...,894322482190.28,768644740597.12,691363412188.46,684898755570.72,767025741475.04,822538393591.45,786460035395.03,844459722152.99,877863284874.07,NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,137244439121.37,87219300384.51,49840491178.15,68972769395.63,77792944471.95,69309110145.77,50241368243.63,65685435100.50,106713618735.43,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,7074392884.05,6295845465.36,6682674062.96,7180768692.26,7878762812.20,7899741278.94,7717143395.26,9412034299.23,9429156201.96,NaN
262,"Yemen, Rep.",YEM,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,43228585321.33,42444489460.94,31317824906.44,26842231204.80,21606161066.21,NaN,NaN,NaN,NaN,NaN
263,South Africa,ZAF,GDP (current US$),NY.GDP.MKTP.CD,8748596504.06,9225996313.29,9813996078.33,10854195662.66,11955995222.38,13068994777.63,...,381198869776.11,346709790458.56,323585509674.48,381448814653.46,404158882459.02,388531226582.35,337619569570.51,419015636064.71,405869718462.34,NaN
264,Zambia,ZMB,GDP (current US$),NY.GDP.MKTP.CD,713000000.00,696285714.29,693142857.14,718714285.71,839428571.43,1082857142.86,...,27141023558.08,21251216798.78,20958412538.31,25873601260.84,26311506435.06,23308667781.23,18110638619.37,22147649568.61,29784454055.94,NaN


In [11]:
gdp_data.drop(columns= ['Country Code', 'Indicator Name', 'Indicator Code'], inplace=True)



In [12]:
gdp_data

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2791061452.51,2963128491.62,2983798882.68,3092178770.95,3276187709.50,3395793854.75,2610038938.55,3126019385.47,NaN,NaN
1,Africa Eastern and Southern,21125015452.20,21616228139.04,23506279899.97,28048360188.28,25920665259.89,29472103269.74,32014368121.37,33269509510.46,36327785494.93,...,1006526390112.83,927348469903.11,885176429223.51,1021043200767.01,1007196197587.41,1000834077088.55,927593321647.66,1081998139192.47,1169483734191.46,NaN
2,Afghanistan,537777811.11,548888895.56,546666677.78,751111191.11,800000044.44,1006666637.78,1399999966.67,1673333417.78,1373333366.67,...,20550582746.84,19998143635.87,18019554403.45,18896353155.88,18418860354.42,18904502222.21,20143451705.75,14583135236.57,NaN,NaN
3,Africa Western and Central,10447637852.92,11173212080.39,11990534017.87,12727688165.45,13898109284.02,14929792387.52,15910837741.97,14510579888.72,14968235781.96,...,894322482190.28,768644740597.12,691363412188.46,684898755570.72,767025741475.04,822538393591.45,786460035395.03,844459722152.99,877863284874.07,NaN
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,137244439121.37,87219300384.51,49840491178.15,68972769395.63,77792944471.95,69309110145.77,50241368243.63,65685435100.50,106713618735.43,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7074392884.05,6295845465.36,6682674062.96,7180768692.26,7878762812.20,7899741278.94,7717143395.26,9412034299.23,9429156201.96,NaN
262,"Yemen, Rep.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43228585321.33,42444489460.94,31317824906.44,26842231204.80,21606161066.21,NaN,NaN,NaN,NaN,NaN
263,South Africa,8748596504.06,9225996313.29,9813996078.33,10854195662.66,11955995222.38,13068994777.63,14211394321.13,15821393677.77,17124793156.93,...,381198869776.11,346709790458.56,323585509674.48,381448814653.46,404158882459.02,388531226582.35,337619569570.51,419015636064.71,405869718462.34,NaN
264,Zambia,713000000.00,696285714.29,693142857.14,718714285.71,839428571.43,1082857142.86,1264285714.29,1368000000.00,1605857142.86,...,27141023558.08,21251216798.78,20958412538.31,25873601260.84,26311506435.06,23308667781.23,18110638619.37,22147649568.61,29784454055.94,NaN


In [13]:
gdp_country_list = pd.DataFrame(data=gdp_data['Country Name'].unique(), columns = ['gdp_country'])
gdp_country_list = gdp_country_list.sort_values(by='gdp_country', ignore_index= True)
gdp_country_list

,gdp_country
0,Afghanistan
1,Africa Eastern and Southern
2,Africa Western and Central
3,Albania
4,Algeria
...,...
261,West Bank and Gaza
262,World
263,"Yemen, Rep."
264,Zambia


## Inflation

In [14]:
# from fred, index 2015, 100
inflation_data = pd.read_csv('data/CPI-data-fred.csv')
inflation_data = inflation_data.rename(columns = {'DATE': 'Period', 'CPALCY01CAM661N':'cpi_val'})
inflation_data['Period'] = pd.to_datetime(inflation_data['Period'])

In [15]:
inflation_data

,Period,cpi_val
0,1988-01-01,56.93
1,1988-02-01,57.33
2,1988-03-01,57.49
3,1988-04-01,57.65
4,1988-05-01,57.97
...,...,...
424,2023-05-01,124.51
425,2023-06-01,125.07
426,2023-07-01,125.23
427,2023-08-01,126.03


# 1. Cleaning
---
---

### Merge 1605 and 0306

In [16]:
# concat function
def concat_dfs(df_list):
    df = pd.concat(df_list, axis = 0, ignore_index = True)
    return df

In [17]:
full_data = concat_dfs([df_1605_full,df_0306_full])


## Cleaning Steps

In [18]:
# perform cleaning set on combined data set
def cleaning_steps(df_name):
    # make lower case
    df_name['Commodity'] = df_name['Commodity'].str.lower()
    # keep only commodities that mention 'lobster'
    df_name = df_name[df_name['Commodity'].str.contains('lobster')].reset_index(drop=True)
    # drop extra columns
    df_name = df_name.drop(columns = ['Unit of measure'])
    # rename columns
    new_col_names = {'Value ($)': 'Value', 'Quantity': 'Quantity_kg'}
    df_name = df_name.rename(columns=new_col_names)
    # update period to datetime object
    df_name['Period'] = pd.to_datetime(df_name['Period'])
    return df_name



In [19]:
data_clean = cleaning_steps(full_data)
data_clean

,Period,Commodity,Province,Country,State,Value,Quantity_kg
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2454203,42059
1,2018-06-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2199419,48158
2,2014-06-01,"1605.30.11 - lobster meat, frozen, cooked by s...",Newfoundland and Labrador,United States,Massachusetts,1299983,38486
3,2014-10-01,"1605.30.11 - lobster meat, frozen, cooked by s...",Newfoundland and Labrador,United States,Massachusetts,151542,3475
4,2014-11-01,"1605.30.11 - lobster meat, frozen, cooked by s...",Newfoundland and Labrador,United States,Massachusetts,634072,22628
...,...,...,...,...,...,...,...
57380,2006-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Northwest Territories,Hong Kong,NaN,6000,273
57381,2007-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Northwest Territories,Japan,NaN,7200,408
57382,2007-03-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Northwest Territories,"Korea, South",NaN,10080,572
57383,2006-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nunavut,Czechia,NaN,3000,136


### Adjust value to today's dollars

In [20]:

def inflation_adjustment(df_name, cpi_df):
    ''' adjust monetary features into today's dolars, index 2015 = 100'''
    df_name = df_name.merge(cpi_df, how='inner', on='Period')
    today_cpi = cpi_df.loc[cpi_df['Period'] == cpi_df['Period'].max(), 'cpi_val'].item()
    # create new column to adjust to today's dollars
    # return df_name
    df_name['Value-adjusted'] = (df_name['Value'] / df_name['cpi_val']) * today_cpi
    return df_name
    
    

In [21]:
data_clean = inflation_adjustment(data_clean, inflation_data)
data_clean

,Period,Commodity,Province,Country,State,Value,Quantity_kg,cpi_val,Value-adjusted
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2454203,42059,102.17,3023516.55
1,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",New Brunswick,United States,Massachusetts,20137016,370393,102.17,24808298.71
2,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,California,31113,1633,102.17,38330.44
3,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Quebec,United States,California,98603,2347,102.17,121476.42
4,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,New Hampshire,310914,15676,102.17,383038.25
...,...,...,...,...,...,...,...,...,...
57380,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",British Columbia,Hong Kong,NaN,81553,4538,59.65,172082.30
57381,1989-02-01,"0306.12.10 - lobsters in brine, (homarus spp),...",Prince Edward Island,Sweden,NaN,107235,8340,59.65,226273.05
57382,1989-02-01,"0306.22.90 - lobsters,(homarus spp), not froze...",Nova Scotia,United States,Massachusetts,16048,1316,59.65,33862.36
57383,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nova Scotia,United States,Unknown states,124924,8381,59.65,263598.02


## Country Name Cleaning

### Clean Country Name in export data first 
before running match

In [22]:
data_clean[["Country", 'Terminated']]= data_clean["Country"].str.split(r'(',expand=True)
data_clean

,Period,Commodity,Province,Country,State,Value,Quantity_kg,cpi_val,Value-adjusted,Terminated
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2454203,42059,102.17,3023516.55,None
1,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",New Brunswick,United States,Massachusetts,20137016,370393,102.17,24808298.71,None
2,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,California,31113,1633,102.17,38330.44,None
3,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Quebec,United States,California,98603,2347,102.17,121476.42,None
4,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,New Hampshire,310914,15676,102.17,383038.25,None
...,...,...,...,...,...,...,...,...,...,...
57380,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",British Columbia,Hong Kong,NaN,81553,4538,59.65,172082.30,None
57381,1989-02-01,"0306.12.10 - lobsters in brine, (homarus spp),...",Prince Edward Island,Sweden,NaN,107235,8340,59.65,226273.05,None
57382,1989-02-01,"0306.22.90 - lobsters,(homarus spp), not froze...",Nova Scotia,United States,Massachusetts,16048,1316,59.65,33862.36,None
57383,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nova Scotia,United States,Unknown states,124924,8381,59.65,263598.02,None


In [23]:
data_clean["Country"] = data_clean["Country"].str.strip()

In [72]:
export_country_list = pd.Series(data_clean['Country'].unique())
export_country_list = export_country_list.sort_values(ignore_index= True)
export_country_list

0              Afghanistan
1                  Albania
2      Antigua And Barbuda
3                Argentina
4                  Armenia
              ...         
153                Uruguay
154             Uzbekistan
155              Venezuela
156               Viet Nam
157             Yugoslavia
Length: 158, dtype: object

In [73]:
export_country_list.to_csv('data/export_countries.csv')

In [25]:
# export clean full data for reuse

data_clean.to_csv('data/data_full_clean.csv')

### Match country names in gdp list to export country list

In [27]:
# filter GDP data by countries in export country list:
gdp_data_filtered = gdp_data[gdp_data['Country Name'].isin(export_country_list)].reset_index(drop=True)
gdp_data_filtered

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2791061452.51,2963128491.62,2983798882.68,3092178770.95,3276187709.50,3395793854.75,2610038938.55,3126019385.47,NaN,NaN
1,Afghanistan,537777811.11,548888895.56,546666677.78,751111191.11,800000044.44,1006666637.78,1399999966.67,1673333417.78,1373333366.67,...,20550582746.84,19998143635.87,18019554403.45,18896353155.88,18418860354.42,18904502222.21,20143451705.75,14583135236.57,NaN,NaN
2,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13228147516.12,11386853143.15,11861199830.84,13019726211.74,15156424061.98,15401826080.52,15162734205.25,17930565118.82,18882095517.88,NaN
3,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,414105366752.89,370275469571.14,369255326235.53,390516804029.95,427049432157.93,417989721742.68,349473015330.16,415021590687.54,507534921715.45,NaN
4,Argentina,NaN,NaN,24450604877.61,18272123664.47,25605249381.76,28344705966.64,28630474727.90,24256667553.26,26436857247.50,...,526319673731.64,594749285413.21,557532317363.45,643628396190.39,524819898586.91,447754686715.08,385540406815.60,487227125385.64,632770284408.51,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,133503867181.97,91030968761.42,93355868977.66,112090503817.40,130891086689.52,153883045524.93,156617719786.25,199765856764.50,160502739236.10,NaN
116,Uruguay,1242289239.20,1547388781.43,1710004407.23,1539681490.78,1975701816.47,1890767155.53,1809185093.55,1597713469.10,1593674184.56,...,61496186973.90,57680328702.24,57480788380.31,65006047680.32,65203071110.60,62048585618.50,53666908479.56,61412268248.95,71177146197.50,NaN
117,United States,543300000000.00,563300000000.00,605100000000.00,638600000000.00,685800000000.00,743700000000.00,815000000000.00,861700000000.00,942500000000.00,...,17550680174000.00,18206020741000.00,18695110842000.00,19477336549000.00,20533057312000.00,21380976119000.00,21060473613000.00,23315080560000.00,25462700000000.00,NaN
118,Uzbekistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80845385816.21,86196264741.94,86138288633.22,62081322741.12,52870108217.56,60283503705.39,60224701295.80,69600614988.60,80391853884.76,NaN


In [54]:
# unmatched countries 
unmatched = list(data_clean.loc[~data_clean['Country'].isin(gdp_data['Country Name']), 'Country'].unique())
unmatched = pd.DataFrame(data=unmatched, columns=['unmatched_export_countries'])

In [55]:
unmatched

,unmatched_export_countries
0,Taiwan
1,Macao
2,Christmas Island
3,United States Minor Outlying Islands
4,"South Africa,"
5,"Moldova,"
6,Central African
7,Saint Pierre And Miquelon
8,Dominican
9,Bahamas


In [30]:
pd.concat([unmatched, gdp_country_list], axis = 1)

,unmatched_export_countries,gdp_country
0,Hong Kong,Afghanistan
1,"Korea, South",Africa Eastern and Southern
2,Taiwan,Africa Western and Central
3,Gambia,Albania
4,Macao,Algeria
...,...,...
261,NaN,West Bank and Gaza
262,NaN,World
263,NaN,"Yemen, Rep."
264,NaN,Zambia


In [31]:
# def country_match(countries_to_match, country_options):
result_df = pd.DataFrame(columns = ['country_to_match','best_match', 'all_matches'] )
for country_index in range(len(unmatched)):
        country_to_match = unmatched.iloc[country_index,0]
        collection = gdp_country_list.iloc[:,0].to_list()
        best_matches = process.extract(country_to_match, collection, scorer=fuzz.ratio)
        new_row = {'country_to_match': country_to_match, 'best_match':best_matches[0][0], 'all_matches':str(best_matches)}
        # print(pd.DataFrame(data = [new_row]))
        result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)

        # result_df = result_df.append(country_to_match, best_matches[0], best_matches, ignore_index = True)
result_df

,country_to_match,best_match,all_matches
0,Hong Kong,"Hong Kong SAR, China","[('Hong Kong SAR, China', 62), ('Mongolia', 47..."
1,"Korea, South","Korea, Rep.","[('Korea, Rep.', 64), ('Lesotho', 53), ('Afric..."
2,Taiwan,Thailand,"[('Thailand', 71), ('Tajikistan', 62), ('Botsw..."
3,Gambia,Zambia,"[('Zambia', 83), ('Namibia', 77), ('Cambodia',..."
4,Macao,Monaco,"[('Monaco', 73), ('Curacao', 67), ('Jamaica', ..."
5,Christmas Island,Cayman Islands,"[('Cayman Islands', 67), ('Channel Islands', 6..."
6,Türkiye,Turkiye,"[('Turkiye', 86), ('Austria', 43), ('Eritrea',..."
7,United States Minor Outlying Islands,British Virgin Islands,"[('British Virgin Islands', 55), ('Northern Ma..."
8,"South Africa, Republic of",Central African Republic,"[('Central African Republic', 69), ('South Afr..."
9,"Moldova, Republic of",Slovak Republic,"[('Slovak Republic', 74), ('Dominican Republic..."


In [32]:

bannedWords = ['republic', 'rep', 'of', 'the', ',']

def RemoveBannedWords(toPrint, banned_words):
    # statement = toPrint
    pattern = re.compile(r'\b(?:' + '|'.join(map(re.escape, banned_words)) + r')\b', re.I)
    return pattern.sub("", toPrint)

# # toPrint = unmatched.loc[unmatched['unmatched_export_countries'].str.contains('tanzania'), 'unmatched_export_countries'].items()
# print(type(toPrint))
# print(toPrint)
# print(RemoveBannedWords(toPrint, bannedWords))

In [33]:
def country_cleaning(df_name, df_column):
    # print(df_name)
    # print(df_name[df_column])
    # df_name.loc[:,df_column] = df_name[df_column].lower()
    df_name[df_column] = df_name[df_column].apply(lambda x: x.lower())
    # df_name[df_column] = df_name[df_column].apply(lambda x: x.replace('of', ''))
    df_name[df_column] = df_name[df_column].apply(lambda x: RemoveBannedWords(x, bannedWords))
    df_name[df_column] = df_name[df_column].apply(lambda x: unidecode(x))
    df_name[df_column] = df_name[df_column].apply(lambda x: x.title())
    return df_name


In [34]:
country_cleaning(data_clean,'Country')


,Period,Commodity,Province,Country,State,Value,Quantity_kg,cpi_val,Value-adjusted,Terminated
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2454203,42059,102.17,3023516.55,None
1,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",New Brunswick,United States,Massachusetts,20137016,370393,102.17,24808298.71,None
2,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,California,31113,1633,102.17,38330.44,None
3,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Quebec,United States,California,98603,2347,102.17,121476.42,None
4,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,New Hampshire,310914,15676,102.17,383038.25,None
...,...,...,...,...,...,...,...,...,...,...
57380,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",British Columbia,Hong Kong,NaN,81553,4538,59.65,172082.30,None
57381,1989-02-01,"0306.12.10 - lobsters in brine, (homarus spp),...",Prince Edward Island,Sweden,NaN,107235,8340,59.65,226273.05,None
57382,1989-02-01,"0306.22.90 - lobsters,(homarus spp), not froze...",Nova Scotia,United States,Massachusetts,16048,1316,59.65,33862.36,None
57383,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nova Scotia,United States,Unknown states,124924,8381,59.65,263598.02,None


In [68]:
# manually update important countries for mapping
gdp_data.loc[gdp_data['Country Name'].str.contains('Korea, Rep.'), 'Country Name'] = 'Korea, South'
gdp_data.loc[gdp_data['Country Name'].str.contains('Hong'), 'Country Name'] = 'Hong Kong'
gdp_data.loc[gdp_data['Country Name'].str.contains('Gambia'), 'Country Name'] = 'Gambia'
gdp_data.loc[gdp_data['Country Name'].str.contains('Egypt'), 'Country Name'] = 'Egypt'
gdp_data.loc[gdp_data['Country Name'].str.contains('Egypt'), 'Country Name'] = 'Macao'
# collapse east /west germany into germany in data clean
data_clean.loc[data_clean['Country'].str.contains('Germany'), 'Country'] = 'Germany'

In [69]:
# def country_match(countries_to_match, country_options):
result_df = pd.DataFrame(columns = ['country_to_match','best_match', 'all_matches'] )
for country_index in range(len(unmatched)):
        country_to_match = unmatched.iloc[country_index,0].title()
        collection = gdp_country_list.iloc[:,0].to_list()
        best_matches = process.extract(country_to_match, collection, scorer=fuzz.ratio)[0]
        new_row = {'country_to_match': country_to_match, 'best_match':best_matches[0], 'all_matches':str(best_matches)}
        # print(pd.DataFrame(data = [new_row]))
        result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)

        # result_df = result_df.append(country_to_match, best_matches[0], best_matches, ignore_index = True)
result_df

,country_to_match,best_match,all_matches
0,Taiwan,Thailand,"('Thailand', 71)"
1,Macao,Monaco,"('Monaco', 73)"
2,Christmas Island,Cayman Islands,"('Cayman Islands', 67)"
3,United States Minor Outlying Islands,British Virgin Islands,"('British Virgin Islands', 55)"
4,"South Africa,",South Africa,"('South Africa', 100)"
5,"Moldova,",Moldova,"('Moldova', 100)"
6,Central African,Central African Republic,"('Central African Republic', 77)"
7,Saint Pierre And Miquelon,Sierra Leone,"('Sierra Leone', 54)"
8,Dominican,Dominica,"('Dominica', 94)"
9,Bahamas,"Bahamas, The","('Bahamas, The', 74)"


In [71]:
# QA checks:
# data_clean.loc[data_clean['Country'].str.contains('Gambia')]
# gdp_data.loc[gdp_data['Country Name'].str.contains('Germany')]
# gdp_data.loc[gdp_data['Country Name'].str.contains('Hong')]
gdp_data.loc[gdp_data['Country Name'].str.contains('Macao')]
# data_clean.loc[data_clean['Country'].str.contains('Germany'), 'Country'].unique()

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
67,Macao,NaN,NaN,NaN,NaN,NaN,"4,948,667,540.41","5,278,005,611.91","5,605,484,298.98","5,932,242,990.65",...,"305,595,408,895.27","329,366,576,819.41","332,441,717,791.41","248,362,771,739.13","262,588,632,526.73","318,678,815,489.75","383,817,841,547.10","424,671,765,455.70","476,747,720,364.74",NaN
146,"Macao SAR, China",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"54,902,831,793.50","45,048,188,247.65","45,070,807,279.63","50,440,935,654.01","55,284,309,690.27","55,204,960,999.59","25,459,685,019.55","30,123,795,336.89","21,979,475,560.62",NaN


In [67]:
unmatched['unmatched_export_countries'] = unmatched['unmatched_export_countries'].str.title()
unmatched

,unmatched_export_countries
0,Taiwan
1,Macao
2,Christmas Island
3,United States Minor Outlying Islands
4,"South Africa,"
5,"Moldova,"
6,Central African
7,Saint Pierre And Miquelon
8,Dominican
9,Bahamas


In [39]:
data_clean.loc[~data_clean['Country'].isin(gdp_data['Country Name']), 'Country'].value_counts()

Country
Taiwan                                          816
Saint Pierre And Miquelon                       222
Korea, North                                     65
Macao                                            36
Guadeloupe                                       26
South Africa,                                    20
Christmas Island                                 11
Martinique                                       10
Czechoslovakia                                   10
Former Union  Soviet Socialist Republics         10
Netherlands Antilles                              9
United States Minor Outlying Islands              9
Trinidad And Tobago                               5
Dominican                                         4
Union  Soviet Socialist Republics                 4
Bahamas                                           3
Iran                                              3
Saint Helena, Ascension And Tristan Da Cunha      3
Bonaire, Sint Eustatius And Saba                  2
Kyrg

In [40]:
# unmatched countries again
unmatched = list(data_clean.loc[~data_clean['Country'].isin(gdp_data_filtered['Country Name']), 'Country'].unique())
unmatched = pd.DataFrame(data=unmatched, columns=['unmatched_export_countries'])
unmatched

,unmatched_export_countries
0,Hong Kong
1,"Korea, South"
2,Taiwan
3,Gambia
4,Macao
5,Christmas Island
6,Turkiye
7,United States Minor Outlying Islands
8,"South Africa,"
9,"Moldova,"


In [41]:
df = data_clean.loc[data_clean['Country'].isin(unmatched['unmatched_export_countries']),:]
df[['Country', 'Value']].groupby('Country').sum().sort_values(by='Value', ascending = False)

,Value
Country,
"Korea, South",1075849587
Hong Kong,644390181
Taiwan,164067902
"Korea, North",5324601
Gambia,2130278
Turkiye,1551305
Saint Pierre And Miquelon,1512152
Macao,1084879
"South Africa,",802516



# 2. EDA on Full Data Set
---
---

In [42]:
# surpress scientific notation
pd.options.display.float_format = '{:,.2f}'.format

In [43]:
full_data['Province'].value_counts()

Province
Nova Scotia                  37769
New Brunswick                21260
Newfoundland and Labrador    16195
British Columbia             14299
Prince Edward Island         13276
Quebec                       10953
Ontario                       3957
Alberta                        213
Manitoba                       108
Northwest Territories           87
Saskatchewan                    57
Nunavut                         13
Yukon                            1
Name: count, dtype: int64

In [44]:
# null values in state are for countries other than US
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118188 entries, 0 to 118187
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Period           118188 non-null  object
 1   Commodity        118188 non-null  object
 2   Province         118188 non-null  object
 3   Country          118188 non-null  object
 4   State            61407 non-null   object
 5   Value ($)        118188 non-null  int64 
 6   Quantity         118188 non-null  int64 
 7   Unit of measure  118188 non-null  object
dtypes: int64(2), object(6)
memory usage: 7.2+ MB


In [45]:
full_data.loc[(full_data['Country'] == 'United States'), 'State'].value_counts()

State
Massachusetts           15574
California               6084
Maine                    4942
New York                 3295
New Hampshire            3278
Washington               3220
Florida                  3196
New Jersey               2892
Rhode Island             2210
Illinois                 1809
Texas                    1156
Connecticut              1148
Virginia                 1127
Georgia                  1081
Oregon                    966
Pennsylvania              960
Hawaii                    866
Nevada                    786
Michigan                  761
Indiana                   676
Maryland                  668
Kentucky                  441
Ohio                      386
Delaware                  382
Unknown states            343
Missouri                  325
North Carolina            320
Tennessee                 299
Colorado                  274
Arizona                   262
South Carolina            259
Minnesota                 201
Louisiana                 199
Alas

In [46]:
exports = data_clean.groupby('Country').mean('Value-adjusted')
exports.sort_values(by='Quantity_kg', ascending=False).head(20)


,Value,Quantity_kg,cpi_val,Value-adjusted
Country,,,,
China,"2,141,458.44","110,934.39",98.90,"2,477,098.72"
United States,"1,112,436.35","45,622.47",86.96,"1,511,284.55"
"Korea, South","791,065.87","37,568.93",93.59,"976,595.98"
Belgium,"584,665.72","34,325.64",87.34,"813,991.73"
Japan,"585,955.01","30,087.70",83.27,"849,206.71"
Netherlands,"508,753.01","29,671.40",87.82,"709,382.99"
France,"379,473.88","28,943.85",85.65,"578,379.93"
Norfolk Island,"154,000.00","25,262.00",79.11,"245,028.34"
Spain,"420,377.82","22,720.22",93.81,"515,795.37"


In [47]:
# store in a list
top_countries = list(exports.sort_values(by='Quantity_kg', ascending=False).head(20).index)
top_countries

['China',
 'United States',
 'Korea, South',
 'Belgium',
 'Japan',
 'Netherlands',
 'France',
 'Norfolk Island',
 'Spain',
 'Viet Nam',
 'Hong Kong',
 'United Kingdom',
 'Italy',
 'Pakistan',
 'Germany',
 'Sierra Leone',
 'Taiwan',
 'French Southern Territories',
 'Denmark',
 'Sweden']

In [48]:
data_clean.groupby('Country').mean('Quantity_kg').sort_values(by='Quantity_kg', ascending = False)

,Value,Quantity_kg,cpi_val,Value-adjusted
Country,,,,
China,"2,141,458.44","110,934.39",98.90,"2,477,098.72"
United States,"1,112,436.35","45,622.47",86.96,"1,511,284.55"
"Korea, South","791,065.87","37,568.93",93.59,"976,595.98"
Belgium,"584,665.72","34,325.64",87.34,"813,991.73"
Japan,"585,955.01","30,087.70",83.27,"849,206.71"
...,...,...,...,...
Cocos,"2,100.00",95.00,85.03,"3,108.47"
Myanmar,855.00,57.00,106.73,"1,008.30"
Tokelau,641.00,48.00,60.53,"1,332.87"


In [49]:
pd.DataFrame(data_clean['Country'].value_counts())

,count
Country,
United States,27364
France,2527
Japan,2245
Hong Kong,1841
Belgium,1828
...,...
"Moldova,",1
"Congo, Democratic",1
Mauritius,1


In [50]:
pd.DataFrame(data_clean['Commodity'].value_counts())

,count
Commodity,
"0306.22.10 - lobsters, (homarus spp), live (terminated 2016-12)",17998
"0306.12.90 - lobsters, (homarus spp), nes, frozen, in shell, including boiled in shell",6824
"0306.12.90 - lobsters, nes, frozen",6379
"0306.32.10 - lobsters, live",5110
"1605.30.11 - lobster meat, frozen, simply boiled in water (terminated 2016-12)",4141
"0306.22.10 - lobsters, live (terminated 2016-12)",4132
"0306.12.10 - lobsters in brine, (homarus spp), frozen, in shell, including boiled in shell",3591
"0306.12.10 - lobsters, in brine, frozen",1676
"1605.30.10 - lobster, prepared or preserved, frozen",1441


In [51]:
filtered_df = data_clean.loc[data_clean['Country'].isin(top_countries),:]
filtered_df

,Period,Commodity,Province,Country,State,Value,Quantity_kg,cpi_val,Value-adjusted,Terminated
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2454203,42059,102.17,"3,023,516.55",None
1,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",New Brunswick,United States,Massachusetts,20137016,370393,102.17,"24,808,298.71",None
2,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,California,31113,1633,102.17,"38,330.44",None
3,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Quebec,United States,California,98603,2347,102.17,"121,476.42",None
4,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,New Hampshire,310914,15676,102.17,"383,038.25",None
...,...,...,...,...,...,...,...,...,...,...
57380,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",British Columbia,Hong Kong,NaN,81553,4538,59.65,"172,082.30",None
57381,1989-02-01,"0306.12.10 - lobsters in brine, (homarus spp),...",Prince Edward Island,Sweden,NaN,107235,8340,59.65,"226,273.05",None
57382,1989-02-01,"0306.22.90 - lobsters,(homarus spp), not froze...",Nova Scotia,United States,Massachusetts,16048,1316,59.65,"33,862.36",None
57383,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nova Scotia,United States,Unknown states,124924,8381,59.65,"263,598.02",None


In [52]:
filtered_df

,Period,Commodity,Province,Country,State,Value,Quantity_kg,cpi_val,Value-adjusted,Terminated
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2454203,42059,102.17,"3,023,516.55",None
1,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",New Brunswick,United States,Massachusetts,20137016,370393,102.17,"24,808,298.71",None
2,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,California,31113,1633,102.17,"38,330.44",None
3,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Quebec,United States,California,98603,2347,102.17,"121,476.42",None
4,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,New Hampshire,310914,15676,102.17,"383,038.25",None
...,...,...,...,...,...,...,...,...,...,...
57380,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",British Columbia,Hong Kong,NaN,81553,4538,59.65,"172,082.30",None
57381,1989-02-01,"0306.12.10 - lobsters in brine, (homarus spp),...",Prince Edward Island,Sweden,NaN,107235,8340,59.65,"226,273.05",None
57382,1989-02-01,"0306.22.90 - lobsters,(homarus spp), not froze...",Nova Scotia,United States,Massachusetts,16048,1316,59.65,"33,862.36",None
57383,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nova Scotia,United States,Unknown states,124924,8381,59.65,"263,598.02",None


In [53]:
cols = ['Period', 'Country', 'Value($)']
filtered_df = filtered_df[['Period','Commodity', 'Country', 'Quantity', 'Value ($)']]


KeyError: "['Quantity', 'Value ($)'] not in index"

In [ ]:
data

In [ ]:
sns.scatterplot(x=data['Period'], y=data['Value ($)'], alpha = 0.2)


# bin

dont need... got relevant HS codes another way

In [ ]:
hs_8 = pd.read_csv('data/codes_hs8.csv', sep = ',',skiprows = 1, engine='python', skipfooter = 5, index_col=False)
# hs_8 = pd.read_csv('codes_hs8.csv', sep = ',', skipfooter = 5)


In [ ]:
pd.set_option('display.max_colwidth', None)
hs_8['Commodity'] = hs_8['Commodity'].str.lower()

In [ ]:
hs_lobster = hs_8.loc[hs_8['Commodity'].str.contains('lobster'),:]

In [ ]:
hs_lobster['Commodity'].unique()

### Testing univariate ARIMA

With only data from 1 country, for 1 commodity, using Quantity.


In [ ]:
# find country with most complete data accross time. most countries have great data since 2017

In [ ]:
data

In [ ]:
codes = ['0306.22.10', '0306.32.10']



In [ ]:
arima_test = filtered_df[filtered_df['Commodity'].str[:10].isin(codes)].sort_values(by='Period')
arima_test.groupby('Country').count().sort_values(by='Period', ascending = False)
# arima_test

In [ ]:
# filter for one country --- belgium as example
arima_test = arima_test.loc[arima_test['Country'] == 'Belgium']

In [ ]:
# drop cextra columns
arima_test.drop(columns=['Country', 'Value ($)'], inplace = True)
arima_test

In [ ]:
# count records by year, check that there is no overlap
arima_test.groupby(arima_test['Period'].dt.year).count()

In [ ]:
# arima_test = arima_test.drop(columns=['Commodity'])
arima_test = arima_test.reset_index(drop=True)
arima_test

In [ ]:
import seaborn as sns

In [ ]:
sns.lineplot(x = arima_test['Period'], y=arima_test['Quantity'])

In [ ]:
from pandas import datetime
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot

def parser(x):
 return datetime.strptime('190'+x, '%Y-%m')

series = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
autocorrelation_plot(series)
pyplot.show()

In [ ]:
japan_data = japan_data[['Period','Quantity']]
japan_data

In [ ]:

japan_data = japan_data.groupby('Period').sum(['Quantity', 'Value ($)']).reset_index(drop=False)
japan_data['Value per kg'] = japan_data['Value ($)'] / japan_data['Quantity']
japan_data

In [ ]:
japan_data['Year'] = japan_data['Period'].dt.year
japan_data



### other

In [ ]:
japan_data_recent = japan_data.loc[japan_data['Period'] > '2020', :].reset_index(drop = True)
japan_data_recent

In [ ]:
sns.lineplot(x=japan_data_recent['Year'], y=japan_data_recent['Quantity'])

In [ ]:
data

In [ ]:
sns.lineplot(x=data['Period'], y=data['Value ($)'])